# R introduksjonskurs - dag 2
## DEL 1
Første leser vi in datasett som vi brukte i dag en og lagre det som en vanlig R dataframe. 


### Lage nye variabler
Her bruker vi tilleggspakke tidyverse for å lage nye variabler. Først må vi kalle inn tidyverse pakken. Here kommer det mange varsling siden flere funksjoner har samme navn som de på SparkR pakke. 

In [ ]:
library(tidyverse)

Vi lager nye variabler ved bruk av `mutate()` funksjonen. 

In [ ]:
df %>% 
    mutate(log_salary = log(salary))

Dataset blir kun lagret i minne om det allokeres

In [ ]:
glimpse(df)

In [ ]:
df <- df %>%
        mutate(log_salary = log(salary))
glimpse(df)

Vi kan også lagre en variable med en ifelse setning

In [ ]:
df <- df %>%
  mutate(salary_kat = ifelse(salary > 25000, "high", "low"))
df

Vi kan bruke `lubridate` for å ta ut år fra en dato.

In [ ]:
library(lubridate)
df <- df %>%
    mutate(aar = year(startdate))
df

Vi kan bruke `as.character()` eller `as.numeric` for å endre variabeltype

In [ ]:
df <- df %>%
    mutate(aar = as.character(aar))
df

## Velge noen rad
For å velge noen rad bruker vi `filter()` funksjonen. Innenfor parentes skriver vi en logisk setning. Flere setninger kan kombinere med komma.

In [ ]:
df %>%
  filter(employee == "John Doe")

Vi kan filtrere data etter en dato interval ved å setteopp et interval første og bruker `%within%`.

In [ ]:
int <- interval(ymd("2008-01-01"), ymd("2020-01-01"))
df %>%
 filter(startdate %within% int)

## Velge noen variabler
For å kun ser på noen variabler bruker vi `select()` funksjonen

In [ ]:
df %>%
    select(employee, aar)

Flere funksjoner kan kombineres ved pipelines (`%>%`).

In [ ]:
df %>%
    filter(employee == "John Doe") %>%
    select(employee, salary, aar)

## Oppsummering/aggregering
Vi bruker `summarise()` funksjonen til å aggregere tall og lage statistikk

In [ ]:
df %>%
    summarise(average_salary = mean(salary))

Det er mulig å kombinere `summarise` med andre funksjoner også.

In [ ]:
df %>%
    filter(salary_kat == "low") %>%
    summarise(average_salary_low = mean(salary),
             antall = n())

## Gruppering
Ofte ønsker vi oppsummering innenfor grupper. Dette kan vi gjøre med `group_by()`


In [ ]:
df %>%
    group_by(salary_kat) %>%
    summarise(average_salary = mean(salary),
             antall = n())

## Rename variable

In [ ]:
df %>%
    rename(dato = aar)

## DEL 2
## Kobling av datasett - justere dette

In [ ]:
library(haven)
trygd <- read_sas("./data/trygd.sas7bdat")


# Endrenavn i trygd datasett
names(trygd)
trygd <- trygd %>%
  rename(region = Region,
         kjonn = Kjonn,
         tid = Tid
  )
glimpse(trygd)

# koble
kommunedata <- left_join(kommunedata, trygd, by = c("region", "tid", "kjonn")) 
glimpse(kommunedata)

## Søylediagram

In [ ]:
kommunedata %>% ggplot(aes(kjonn)) + 
  geom_bar()

Her trenger vi å spesifisere `stat="identify"` så at det brukes verdiene istedenfor å telle

In [ ]:
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers)) +
  geom_bar(stat = "identity")

Vi kan endre format på akser ved `scale_y_continuous()`

In [ ]:
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(labels = scales::comma)

Også viser forskjellige år med forskjellige farge ved bruk av `fill=` i `aes()`.

In [ ]:
kommunedata %>%
  ggplot(aes(x = kjonn, y = UforetygdPers, fill = tid)) +
  geom_bar(stat = "identity") +
  scale_y_continuous(labels = scales::comma) 

In [ ]:
For å viser dem ved siden av hverandre bruker vi `position = "dodge"`.

In [ ]:
kommunedata %>%
  filter(region == "0402") %>% 
  ggplot(aes(x = kjonn, y = UforetygdPers, fill = tid)) +
  geom_bar(stat = "identity", position = "dodge") + 
  scale_y_continuous(labels = scales::comma) 

In [ ]:






#### Punktdiagram ####
kommunedata %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) 

# Filter for kun kommune-nivå data
kommunedata %>%
  filter(region_type == "kommune") %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) 

# Legg til regresjonslinje med geom_smotth
kommunedata %>%
  filter(region_type == "kommune") %>%
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point() + 
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma) +
  geom_smooth(method = "lm")

# Farge etter kjønn
kommunedata %>%
  filter(region_type == "kommune") %>% 
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point(aes(color = kjonn)) +
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma)

# Gir akser et nytt navn
kommunedata %>%
  filter(region_type == "kommune") %>% 
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point(aes(color = kjonn)) +
  scale_y_continuous(labels = scales::comma, name = "Brutto inntekt") +
  scale_x_continuous(labels = scales::comma, name = "Antall personer")



#### Lagre figur ####
png(file = "folkemengde_inntekt.png")

kommunedata %>%
  filter(region_type == "kommune") %>% 
  ggplot(aes(x = folkemengde, y = bruttoinnt)) + 
  geom_point(aes(color = kjonn)) +
  scale_y_continuous(labels = scales::comma) +
  scale_x_continuous(labels = scales::comma)

dev.off()




#### Eksportere data/en tabell til excel ####
library(openxlsx)
write.xlsx(kommunedata, file = "kommune_data.xlsx")

https://spark.rstudio.com/